In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# DBSCAN

In [ ]:
customers_ml_data = pd.read_csv('data/online_retail_afterEDA.csv', index_col='CustomerID')
non_binary_cols = [
    'balance', 'max_spent', 'mean_spent', 
    'min_spent', 'n_orders','total_items', 
    'total_refunded', 'total_spent' ]

customers = customers_ml_data[non_binary_cols]
Xscores = pd.read_csv('data/pca_scores.csv', index_col='CustomerID')

In [ ]:
# This function generates a pairplot enhanced with the result of k-means
def pairplot_cluster(df, cols, cluster_assignment):
    """
    Input
        df, dataframe that contains the data to plot
        cols, columns to consider for the plot
        cluster_assignments, cluster asignment returned 
        by the clustering algorithm
    """
    # seaborn will color the samples according to the column cluster
    df['cluster'] = cluster_assignment 
    sns.pairplot(df, vars=cols, hue='cluster')
    df.drop('cluster', axis=1, inplace=True)

The DBSCAN algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are globular. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, min_samples and eps, which define formally what we mean when we say dense. Higher min_samples or lower eps indicate higher density necessary to form a cluster. 

Summary of the Algorithm:

- starts with an arbitrary starting point and retrieved all the points in the radius of distance `eps` from it 
    - if the radius contains `min_samples` points, start a cluster
      - add all the points in the radius of distance `eps` to the cluster and their `eps` neighbors.
      - continue expanding the cluster iterating on the the procedure on all the neighbors
    - otherwise mark it as noise/outlier

Sklearn implementation doc: http://scikit-learn.org/stable/modules/clustering.html#dbscan

Animated DBSCAN: http://www.naftaliharris.com/blog/visualizing-dbscan-clustering/

## Learning Activity - A starting value for eps

Measure the distance of each point to its closest neighbor using the function `sklearn.metrics.pairwise.pairwise_distances` (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) and plot the distribution of the distances.

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Compute all the pairwise distances
all_distances = pairwise_distances(customers, metric='euclidean')


In [ ]:
# compute the distance of each point to its closest neighbor
neig_distances = [np.min(row[np.nonzero(row)]) for row in all_distances]


In [ ]:
# Plot the distances
plt.hist(neig_distances, bins=50)
plt.xlabel('Distance from closest sample')
plt.ylabel('Occurrences')
plt.axis([0,1.5,0,2500])


The distribution of the distance will help us choose a starting point for `eps`. We see that it's very likely that a point as at least one neighbour in a radius of 0.15 and that only very few point have one at distance over 1.0. Since we want that a core point has more than one point in is `eps`-neighborhood we can start picking `eps` on the right tail of the distribution.

## Learning Activity - Applying DBSCAN

Cluster the customer data with DBSCAN and visualise the results in the subspaces used for the other algorithms.

In [ ]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN setting eps to 1.0 and min samples to 8 (say)

db = DBSCAN(eps=1.0, min_samples=8)
cluster_assignment = db.fit_predict(customers)


In [ ]:
# Display how many clusters were found

clusters_found = np.unique(cluster_assignment)
print ('Clusters found', len(clusters_found))


We can then visualise this

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)


DBSCAN clustered all the points in one big cluster and marked as outiers all the points that are not in dense areas.

### Learning Activity -  How many clusters with DBSCAN?

Vary `eps` and `min_samples` and study how the number of clusters varies as result. This way we'll have an idea of how many cluster we get varying the parameters. This can help us choose the parameters if we already have an idea of how many clusters we want to create.

Warning, if you cover a grid of points, it may take a while to finish, don't put too many points!

In [ ]:
# WARNING this may take a couple of minutes to finish!
eps = np.linspace(.5, 10.0, 5)
mins = np.arange(5, 50, 5)
Z = np.zeros((len(eps), len(mins)))

for i, e in enumerate(eps):
    for j, m in enumerate(mins):
        db   = DBSCAN(eps=e, min_samples=m)
        pred = db.fit_predict(customers)
        clusters_found = len(np.unique(pred))
        Z[i,j] = clusters_found

In [ ]:
# Visualise this using a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(Z, cmap='RdBu', center=0, annot=True);
plt.xticks(np.arange(Z.shape[1]), mins)
plt.xlabel('min_samples')
plt.yticks(np.arange(Z.shape[0]), ['%0.2f' % x for x in eps])
plt.ylabel('eps')


### Learning activity - Compute the silhouette score of the DBSCAN cluster

Compute the silhouette score of the clusters made with DBSCAN and compare it with the silhouette score achieved with K-Means.

In [ ]:
# Compute the silhouette score of DBSCAN
print(silhouette_score(customers, cluster_assignment))


### Test Activity - Run DBSCAN on the dataset obtained with PCA

Run DBSCAN on the first 3 principal components in `Xscores`.

Tweak the parameters to achieve about 5 clusters as result.

In [ ]:
# Run DBSCAN on the first 3 principal components

db = DBSCAN(eps=.9, min_samples=5)
cluster_assignment = db.fit_predict(Xscores[['PC1', 'PC2', 'PC3']])

clusters_found = np.unique(cluster_assignment)
print ('Clusters found', len(clusters_found))

pairplot_cluster(Xscores, ['PC1', 'PC2', 'PC3'], cluster_assignment )
